In [1]:
!pip install unidecode
!pip install keras_nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 22.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.5/337.5 KB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 90.9 MB/s eta 0:00:00


In [1]:
import os
import numpy as np

np.random.seed(1234)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import re
import os
import tensorflow as tf 
import unidecode
import numpy as np

from tensorflow.keras import Model
from tensorflow.keras.layers import Input

error_rate = 0.8
hidden_size = 512
nb_epochs = 100
train_batch_size = 512
val_batch_size = 256
sample_mode = 'argmax'

reverse = True

In [2]:
SOS = '\t' # start of sequence.
EOS = '*' # end of sequence.
CHARS = list('abcdefghijklmnopqrstuvwxyz')
REMOVE_CHARS = '[#$%"\+@<=>!&,-.?:;()*\[\]^_`{|}~/\d\t\n\r\x0b\x0c]'

In [3]:
def add_speling_erors(token, error_rate):
    """Simulate some artificial spelling mistakes."""
    assert(0.0 <= error_rate < 1.0)
    if len(token) < 3:
        return token
    rand = np.random.rand()
    # Here are 4 different ways spelling mistakes can occur,
    # each of which has equal chance.
    prob = error_rate / 4.0
    if rand < prob:
        # Replace a character with a random character.
        random_char_index = np.random.randint(len(token))
        token = token[:random_char_index] + np.random.choice(CHARS) \
                + token[random_char_index + 1:]
    elif prob < rand < prob * 2:
        # Delete a character.
        random_char_index = np.random.randint(len(token))
        token = token[:random_char_index] + token[random_char_index + 1:]
    elif prob * 2 < rand < prob * 3:
        # Add a random character.
        random_char_index = np.random.randint(len(token))
        token = token[:random_char_index] + np.random.choice(CHARS) \
                + token[random_char_index:]
    elif prob * 3 < rand < prob * 4:
        # Transpose 2 characters.
        random_char_index = np.random.randint(len(token) - 1)
        token = token[:random_char_index]  + token[random_char_index + 1] \
                + token[random_char_index] + token[random_char_index + 2:]
    else:
        # Replace a character with a random character.
        random_char_index = np.random.randint(len(token))
        token = token[:random_char_index] + np.random.choice(CHARS) \
                + token[random_char_index + 1:]
                
    return token


In [4]:
def transform(tokens, maxlen, error_rate=0.3, shuffle=True):
    """Transform tokens into model inputs and targets.
    All inputs and targets are padded to maxlen with EOS character.
    """
    if shuffle:
        print('Shuffling data.')
        np.random.shuffle(tokens)
    encoder_tokens = []
    decoder_tokens = []
    target_tokens = []
    for token in tokens:
        encoder = add_speling_erors(token, error_rate=error_rate)
        encoder += EOS * (maxlen - len(encoder)) # Padded to maxlen.
        encoder_tokens.append(encoder)
    
        decoder = SOS + token
        decoder += EOS * (maxlen - len(decoder))
        decoder_tokens.append(decoder)
    
        target = decoder[1:]
        target += EOS * (maxlen - len(target))
        target_tokens.append(target)
        
        assert(len(encoder) == len(decoder) == len(target))

    for token in tokens:
        encoder = add_speling_erors(token, error_rate=error_rate)
        encoder += EOS * (maxlen - len(encoder)) # Padded to maxlen.
        encoder_tokens.append(encoder)
    
        decoder = SOS + token
        decoder += EOS * (maxlen - len(decoder))
        decoder_tokens.append(decoder)
    
        target = decoder[1:]
        target += EOS * (maxlen - len(target))
        target_tokens.append(target)
        
        assert(len(encoder) == len(decoder) == len(target))

    for token in tokens:
        encoder = token
        encoder += EOS * (maxlen - len(encoder)) # Padded to maxlen.
        encoder_tokens.append(encoder)
        
        decoder = SOS + token
        decoder += EOS * (maxlen - len(decoder))
        decoder_tokens.append(decoder)
    
        target = decoder[1:]
        target += EOS * (maxlen - len(target))
        target_tokens.append(target)
        
        assert(len(encoder) == len(decoder) == len(target))
    return encoder_tokens, decoder_tokens, target_tokens

In [5]:
with open('30k.txt', 'r') as file:
    vocab = file.readlines()

vocab=[word.strip() for word in vocab if len(word)>1]

In [6]:
with open('10k.txt', 'r') as file:
    val_tokens = file.readlines()

val_tokens=[word.strip() for word in val_tokens if len(word)>1]

In [7]:
BATCH_SIZE = 512
EPOCHS = 1  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 40
ENG_VOCAB_SIZE = 30000
SPA_VOCAB_SIZE = 30000

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

In [8]:
vocab = list(filter(None, set(vocab)))

# `maxlen` is the length of the longest word in the vocabulary
# plus two SOS and EOS characters.
maxlen = max([len(token) for token in vocab]) + 2
train_encoder, train_decoder, train_target = transform(
    vocab, maxlen, error_rate=error_rate, shuffle=False)
print(train_encoder[:10])
print(train_decoder[:10])
print(train_target[:10])

input_chars = set(' '.join(train_encoder))
target_chars = set(' '.join(train_decoder))
nb_input_chars = len(input_chars)
nb_target_chars = len(target_chars)

print('Size of training vocabulary =', len(vocab))
print('Number of unique input characters:', nb_input_chars)
print('Number of unique target characters:', nb_target_chars)
print('Max sequence length in the training set:', maxlen)

# Prepare validation data.
val_tokens = list(filter(None, val_tokens))

val_maxlen = max([len(token) for token in val_tokens]) + 2
val_encoder, val_decoder, val_target = transform(
    val_tokens, maxlen, error_rate=error_rate, shuffle=False)
print(val_encoder[:10])
print(val_decoder[:10])
print(val_target[:10])
print('Number of non-unique validation tokens =', len(val_tokens))
print('Max sequence length in the validation set:', val_maxlen)

['placemvnt******************', 'zwsj***********************', 'allergy********************', 'masn***********************', 'wps************************', 'esigner********************', 'jamount********************', 'guqrdians******************', 'gardicner******************', 'iniiatives*****************']
['\tplacement*****************', '\twsj***********************', '\tallergy*******************', '\tmason*********************', '\tips***********************', '\tdesigner******************', '\tamount********************', '\tguardians*****************', '\tgardiner******************', '\tinitiatives***************']
['placement******************', 'wsj************************', 'allergy********************', 'mason**********************', 'ips************************', 'designer*******************', 'amount*********************', 'guardians******************', 'gardiner*******************', 'initiatives****************']
Size of training vocabulary = 30000
Number of unique inpu

In [9]:
# Loop through each character in the string and encode it
def encoding(string):
  encoded_string = []
  for character in string:
      encoded_string.append(ord(character))
  return np.array(encoded_string)

print(encoding("hello"))

[104 101 108 108 111]


In [10]:
def batch(tokens, maxlen, ctable, batch_size=128, reverse=False):
  data_batchs=[]
  for token in tokens:
    data_batch = encoding(token)
    data_batchs.append(data_batch)
  return data_batchs

In [11]:
train_encoder_batch = batch(train_encoder, maxlen,
                            train_batch_size, reverse)  

train_decoder_batch = batch(train_decoder, maxlen,
                            train_batch_size)
train_target_batch  = batch(train_target, maxlen,
                            train_batch_size)   
 
val_encoder_batch = batch(val_encoder, maxlen,
                          val_batch_size, reverse)
val_decoder_batch = batch(val_decoder, maxlen,
                          val_batch_size)
val_target_batch  = batch(val_target, maxlen,
                          val_batch_size)

In [12]:
def preprocess_batch(train_encoder_batch, train_decoder_batch,train_target_batch):
    return (
        {
            "encoder_inputs": train_encoder_batch,
            "decoder_inputs": train_decoder_batch,
        },
        train_target_batch,
    )


def make_dataset(train_encoder_batch, train_decoder_batch,train_target_batch):

    dataset = tf.data.Dataset.from_tensor_slices((train_encoder_batch, train_decoder_batch,train_target_batch))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_encoder_batch, train_decoder_batch,train_target_batch)
val_ds = make_dataset(val_encoder_batch,val_decoder_batch,val_target_batch)

In [13]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")
    print(inputs["decoder_inputs"][0])

inputs["encoder_inputs"].shape: (512, 27)
inputs["decoder_inputs"].shape: (512, 27)
targets.shape: (512, 27)
tf.Tensor(
[  9 103 114  97 112 101  42  42  42  42  42  42  42  42  42  42  42  42
  42  42  42  42  42  42  42  42  42], shape=(27,), dtype=int64)


In [14]:
# Encoder

BATCH_SIZE = 256
EPOCHS = 1  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 30

EMBED_DIM = 512
INTERMEDIATE_DIM = 512
NUM_HEADS = 16

import keras 
import keras_nlp

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=50000,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(encoder_inputs)
x = keras.layers.Dropout(0.3)(x)
encoder_outputs=x

for _ in range(2):
  encoder_outputs = keras_nlp.layers.TransformerEncoder(
        intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
    )(inputs=encoder_outputs)
  encoder_outputs = keras.layers.Dropout(0.3)(encoder_outputs)

encoder = keras.Model(encoder_inputs, encoder_outputs)

# Decoder
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=50000,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(decoder_inputs)

for _ in range(2):
  x = keras_nlp.layers.TransformerDecoder(
        intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
    )(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
  x = keras.layers.Dropout(0.3)(x)

decoder_outputs = keras.layers.Dense(150, activation="softmax")(x)
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="transformer",
)


transformer.compile(
    "adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)


In [ ]:
# Train and evaluate.
no_epochs=500
for epoch in range(no_epochs):
    print('Main Epoch {:d}/{:d}'.format(epoch + 1, no_epochs))

    train_encoder, train_decoder, train_target = transform(
        vocab, maxlen, error_rate=error_rate, shuffle=True)
    
    train_encoder_batch = batch(train_encoder, maxlen,
                            train_batch_size, reverse)  

    train_decoder_batch = batch(train_decoder, maxlen,
                                train_batch_size)
    train_target_batch  = batch(train_target, maxlen,
                                train_batch_size)   
    
    val_encoder_batch = batch(val_encoder, maxlen,
                              val_batch_size, reverse)
    val_decoder_batch = batch(val_decoder, maxlen,
                              val_batch_size)
    val_target_batch  = batch(val_target, maxlen,
                              val_batch_size)
    train_ds = make_dataset(train_encoder_batch, train_decoder_batch,train_target_batch)
    val_ds = make_dataset(val_encoder_batch,val_decoder_batch,val_target_batch)

    transformer.fit(train_ds, epochs=1, validation_data=val_ds)


Main Epoch 1/500
Shuffling data.
272/352 [======================>.......] - ETA: 31s - loss: 1.4163 - accuracy: 0.7416

In [ ]:
def encoding_input(string):
  encoded_string = []
  length=len(string)

  total_pending=27-length
  string=list(string)+[EOS]*total_pending
  for character in string:
      encoded_string.append(ord(character))
  # Convert the list to a tensor
  encoded_string = tf.convert_to_tensor(encoded_string)

  # Reshape the tensor to have shape (1, 27)
  encoded_string = tf.reshape(encoded_string, [1, 27])
  return encoded_string

tokenized_input_sentence=encoding_input("gamng")
tokenized_input_sentence


tokenized_input_sentence

In [ ]:
def inference(word):
  decoder_input_tokens=encoding_input(SOS)
  tokenized_input_sentence=encoding_input(word)
  result=''
  for i in range(maxlen):
    predictions=transformer([tokenized_input_sentence, decoder_input_tokens])
    sampled_token_index = np.argmax(predictions[0, i, :])
    decoder_input_tokens = tf.tensor_scatter_nd_update(decoder_input_tokens, [[0, i+1]], [sampled_token_index])
    
    if sampled_token_index==42:
      break
    result=result+chr(sampled_token_index)
  return result

inference("informtion")